In [1]:
import pandas as pd
import numpy as np
import csv
from copy import deepcopy
from pandas import datetime
from copy import deepcopy
import gc
import random
import datetime
import csv
from copy import deepcopy
from os import listdir
from os.path import isfile, join, isdir
from tqdm import tqdm_notebook
from Downloader import Extractor
from Preprocessing import Preprocessor, set_onelevel, set_multilevel
from Filters import XGB_Quantile
from Filters import quantile_huber_loss, quantile_huber_obj 
import pickle
import xgboost as xgb
from Solar import AddSolarAngles
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Loading Data

In [2]:
# getting file tree in directory "data_csv", which contains raw unfiltered data
mypath = '/home/ndsviriden/data_csv/Raw'
dateparse = pd.to_datetime
station_files = [f for f in listdir(mypath) if isfile(join(mypath, f)) if int(f.split('_', 1)[0])]
station_files.sort()

# reading loaded csv files from data_csv directory, output is a list (length=number of stations) of raw df 
raw_lmeteo_list = [pd.read_csv(mypath + '/' + station, index_col=0, 
                    dtype={'station_id': int,'date_time': str}, 
                    date_parser = dateparse, parse_dates = ['date_time']) 
                   for station in tqdm_notebook(station_files[:50])]

#fix problems with indexing, later this will be solved in loading part
for elem in raw_lmeteo_list:
    elem.reset_index(drop=True, inplace=True)

raw_data = pd.concat(raw_lmeteo_list)
raw_data['id'] = random.sample(range(len(raw_data)), len(raw_data))
raw_data['valid'] = True
raw_data = raw_data.reset_index(drop=True)

del raw_lmeteo_list
gc.collect()

raw_data

,station_data_id,sensor_id,data,station_id,sensor_type_id,active_state,name,rule_id,rule_ordr,direction,lane_traffic,date_time,poll_interval,fake_data,type,unit_of_measure,description,id,valid
0,7137045.0,6032.0,7357.0,113,16.0,A,NaN,152.0,36.0,0.0,0.0,2012-09-17 18:01:48,NaN,NaN,pressure,NaN,pressure (sw),7308638,True
1,7137045.0,6021.0,140.0,113,2.0,A,NaN,154.0,14.0,0.0,0.0,2012-09-17 18:01:48,NaN,NaN,t_road,NaN,t_road (sw),12910059,True
2,7137045.0,6020.0,136.0,113,1.0,A,NaN,4.0,12.0,0.0,0.0,2012-09-17 18:01:48,NaN,NaN,t_air,NaN,t_air (sw),9336779,True
3,7137045.0,6022.0,140.0,113,3.0,A,NaN,155.0,16.0,0.0,0.0,2012-09-17 18:01:48,NaN,NaN,t_underroad,NaN,t_underroad (sw),8739149,True
4,7137045.0,6023.0,831.0,113,4.0,A,NaN,6.0,18.0,0.0,0.0,2012-09-17 18:01:48,NaN,NaN,dampness,NaN,dampness (sw),1223262,True
5,7137243.0,6032.0,7398.0,113,16.0,A,NaN,152.0,36.0,0.0,0.0,2012-09-18 14:33:21,NaN,NaN,pressure,NaN,pressure (sw),5406387,True
6,7137243.0,6021.0,194.0,113,2.0,A,NaN,154.0,14.0,0.0,0.0,2012-09-18 14:33:21,NaN,NaN,t_road,NaN,t_road (sw),6615481,True
7,7137243.0,6020.0,156.0,113,1.0,A,NaN,4.0,12.0,0.0,0.0,2012-09-18 14:33:21,NaN,NaN,t_air,NaN,t_air (sw),7790392,True
8,7137243.0,6022.0,187.0,113,3.0,A,NaN,155.0,16.0,0.0,0.0,2012-09-18 14:33:21,NaN,NaN,t_underroad,NaN,t_underroad (sw),12434628,True
9,7137243.0,6023.0,664.0,113,4.0,A,NaN,6.0,18.0,0.0,0.0,2012-09-18 14:33:21,NaN,NaN,dampness,NaN,dampness (sw),13258695,True


In [3]:
time_gap = pd.Timedelta('5d')
preprocessor = Preprocessor()
meteo_split = preprocessor.PivotTable(raw_data)
meteo_split = preprocessor.FixPressureScale(meteo_split)
patterns_list = preprocessor.CreatePatternList(meteo_split, max_gap=time_gap)

In [159]:
test = deepcopy(patterns_list[3])
mask = test.date_time >= pd.Timestamp(2016, 1, 1)
test[('data', 'pressure')][mask] = np.nan
test[('data', 't_air')].iloc[10:25] = np.nan
test[('data', 't_air')].iloc[35:45] = np.nan
print(len(test))

63935


In [176]:
z = pd.concat((test.date_time, test.data['t_air']), axis=1)
z.index.diff()

AttributeError: 'Int64Index' object has no attribute 'diff'

In [172]:
def round_30min(time):
    return datetime.datetime(time.year, time.month, time.day, time.hour, time.minute - time.minute % 30, 0)

def make_grid(df):
    df_result = deepcopy(df)
    start = round_30min(df_result.date_time.min())
    end = round_30min(df_result.date_time.max())

    df_add = pd.DataFrame(pd.date_range(start=start, end=end, freq='30min'),
                          columns=pd.MultiIndex.from_tuples([('date_time', '')]))
    
    df_add[('station_id', '')] = df_result.station_id.unique()[0]

    # adding a new column, so that we can define, which data is original and which is interpolated
    df_result[('interpol', '')] = False
    df_add[('interpol', '')] = True

    # Merging 2 tables
    df_result = pd.merge(df_result, df_add, how='outer',
                         on=['date_time', 'interpol', 'station_id'])

    df_result.valid = True
    df_result = df_result.set_index('date_time')
    df_result = df_result.sort_index()
    #df_result = df_result.reset_index(drop=False)
    return df_result
    

def interpolate(pattern, limit=pd.Timedelta('2h')):
    df_result = deepcopy(pattern)
    df_result = df_result.reset_index()
    df_result['time_delta'] = df_result.date_time.diff()
    for sensor in test.data.columns:
        df = pd.concat((df_result.date_time, df_result.time_delta, df_result.data[sensor]), axis=1)
        df['group'] = (df[sensor].notnull() != df[sensor].shift().notnull()).cumsum() #consecutive nan/not nan
        df['consecutive'] = (df[sensor].notnull() != df[sensor].shift(-1).notnull()).apply(lambda x: float(x))
        df['gap'] = df[sensor].isnull() * df['date_time'][df['consecutive']==1].diff()
        df['gap'][df['gap'].isnull()] = 0
        groups_to_consider = list(df['group'][df['gap'] > limit].values)
        mask = df['group'].isin(groups_to_consider)
        df['gap'][mask] = df[mask].groupby('group')['time_delta'].apply(lambda g: g.cumsum())
        
        df = df.set_index('date_time')
        mask = df['gap'] <= limit
        df[sensor][mask] = df[sensor].interpolate(method='time')[mask]
        df = df.reset_index()
        if sensor=='t_air':
            z = df
            
        #mask = df['group_diff'] <= limit
        #df[sensor][mask] = df[sensor].interpolate(method='time')[mask]
        #df_result[('data', sensor)] = deepcopy(df[sensor])
    #del df_result['time_delta']
    #return df_result
    return z

In [173]:
%%time
z = interpolate(make_grid(test))
#x = z['group'].value_counts()
#groups_to_use = list(x[x > 2].index.values)
#z[z['group'].isin(groups_to_use)]

CPU times: user 6.2 s, sys: 108 ms, total: 6.31 s
Wall time: 6.4 s


In [153]:
pd.set_option("display.max_rows",200)
display(z)

,date_time,time_delta,t_air,group,consecutive,gap
0,2012-09-17 18:00:00,NaT,NaN,0,1.0,00:00:00
1,2012-09-17 18:02:10,00:02:10,142.000000,1,1.0,00:00:00
2,2012-09-17 18:30:00,00:27:50,139.236624,2,1.0,00:27:50
3,2012-09-17 18:32:23,00:02:23,139.000000,3,1.0,00:00:00
4,2012-09-17 19:00:00,00:27:37,139.000000,4,1.0,00:27:37
5,2012-09-17 19:02:35,00:02:35,139.000000,5,1.0,00:00:00
6,2012-09-17 19:30:00,00:27:25,139.906836,6,1.0,00:27:25
7,2012-09-17 19:32:49,00:02:49,140.000000,7,0.0,00:00:00
8,2012-09-17 19:37:14,00:04:25,139.000000,7,1.0,00:00:00
9,2012-09-17 20:00:00,00:22:46,139.753447,8,1.0,00:22:46


In [290]:
panda_array = np.array([1, 2, 3, np.nan, np.nan, np.nan, np.nan, 8, 9, np.nan, 
                        11, np.nan, 15, 16, 17])
panda = pd.DataFrame(panda_array, columns=['value'])
panda = panda.reset_index()
panda['diff'] = panda.value.notnull() != panda.value.shift(-1).notnull()
panda['diff'] = panda['diff'].apply(lambda x: float(x))
panda['group'] = (panda.value.notnull() != panda.value.shift().notnull()).cumsum()
panda['gap'] = panda['value'].isnull() * panda['index'][panda['diff']==1].diff()
panda['gap'][panda['gap'].isnull()] = 0
panda['gap'][panda['gap'] < 2] = 0
groups_to_consider = list(panda['group'][panda['gap'] > 0].values)
panda[panda['group'].isin(groups_to_consider)]

,index,value,diff,group,gap
3,3,NaN,0.0,2,0.0
4,4,NaN,0.0,2,0.0
5,5,NaN,0.0,2,0.0
6,6,NaN,1.0,2,4.0


In [79]:
def round_30min(time):
    return datetime.datetime(time.year, time.month, time.day, time.hour, time.minute - time.minute % 30, 0)

def interpolate(df):
    # copy the initial dataframe, so that no actions inside the function can affect it
    df_result = deepcopy(df)

    start = round_30min(df_result.date_time.min())
    end = round_30min(df_result.date_time.max())

    df_add = pd.DataFrame(pd.date_range(start=start, end=end, freq='30min'),
                          columns=pd.MultiIndex.from_tuples([('date_time', '')]))

    # !!if len(df_result.station_id.unique()) > 1 --- Raise Error ##
    df_add[('station_id', '')] = df_result.station_id.unique()[0]

    # adding a new column, so that we can define, which data is original and which is interpolated
    df_result[('interpol', '')] = False
    df_add[('interpol', '')] = True

    # Merging 2 tables
    df_result = pd.merge(df_result, df_add, how='outer',
                         on=['date_time', 'interpol', 'station_id'])

    df_result = df_result.set_index('date_time')
    df_result = df_result.sort_index()
    df_result.valid = True

    # Interpolating data in '30min - round' timestamp
    df_result.data = df_result.data.interpolate(method='time', limit_direction='both')
    df_result.id = df_result.id.interpolate(method='nearest', limit_direction='both')

    # Deleting original data, leaving only interpolated rows
    df_result = df_result[df_result.interpol]
    del df_result[('interpol',)]
    df_result = df_result.dropna()
    df_result = df_result.reset_index()
    return df_result

In [43]:
mask = df_result.interpol
df_result.interpolate(method='time', limit_direction='both')[mask]

data                                       \
                        dampness     pressure       t_air      t_road   
date_time                                                               
2012-09-17 18:00:00   741.000000  7406.000000  142.000000  152.000000   
2012-09-17 18:30:00   760.343629  7406.000000  139.236624  146.473249   
2012-09-17 19:00:00   770.230132  7406.000000  139.000000  143.256623   
2012-09-17 19:30:00   766.465821  7406.000000  139.906836  143.000000   
2012-09-17 20:00:00   753.944843  7412.027579  139.753447  142.246553   
2012-09-17 20:30:00   752.238831  7414.000000  136.268616  139.761169   
2012-09-17 21:00:00   758.173745  7414.000000  132.043574  136.782681   
2012-09-17 21:30:00   735.832781  7414.000000  133.197020  135.267660   
2012-09-17 22:00:00   726.275881  7420.517070  134.000000  135.724119   
2012-09-17 22:30:00   715.258545  7416.555127  135.432194  136.716097   
2012-09-17 23:00:00   713.836645  7420.382450  135.290839  137.000000   
2012-09-17 23:30:00   710.784768  7416.677152  134.297461  137.000000   
2012-09-18 00:00:00   711.768386  7420.228869  133.307903  136.307903   
2012-09-18 00:30:00   717.089109  7423.000000  130.273927  133.955446   
2012-09-18 01:00:00   725.064570  7423.000000  123.609272  128.956954   
2012-09-18 01:30:00   750.672918  7423.000000  115.665196  120.331495   
2012-09-18 02:00:00   778.492002  7423.000000  109.701600  112.382239   
2012-09-18 02:30:00   798.745174  7423.000000  105.389961  106.737452   
2012-09-18 03:00:00   814.261214  7423.000000  102.765172  103.147757   
2012-09-18 03:30:00   836.556656  7423.000000   99.043454   99.634763   
2012-09-18 04:00:00   860.223018  7427.656388   95.253855   95.671806   
2012-09-18 04:30:00   874.745033  7426.403974   92.850993   92.276490   
2012-09-18 05:00:00   875.594595  7423.000000   92.567568   90.432432   
2012-09-18 05:30:00   881.971302  7427.485651   90.196468   88.878587   
2012-09-18 06:00:00   885.675497  7431.000000   88.000000   86.891832   
2012-09-18 06:30:00   875.335541  7431.000000   90.737307   89.832230   
2012-09-18 07:00:00   857.648649  7431.000000   95.162162  101.648649   
2012-09-18 07:30:00   841.372173  7431.000000   98.325565  116.290610   
2012-09-18 08:00:00   825.735245  7431.000000  104.050193  129.378930   
2012-09-18 08:30:00   798.844923  7436.717439  112.432119  143.005519   
...                          ...          ...         ...         ...   
2016-07-13 05:00:00  1000.000000   744.000000  140.494075  181.144029   
2016-07-13 05:30:00  1000.000000   744.000000  149.245214  199.193254   
2016-07-13 06:00:00   995.493421   744.000000  156.703947  215.759868   
2016-07-13 06:30:00   968.988481   744.000000  167.003840  237.569940   
2016-07-13 07:00:00   917.243056   744.000000  179.659722  254.138889   
2016-07-13 07:30:00   859.743056   744.000000  188.159722  252.138889   
2016-07-13 08:00:00   726.117097   744.000000  211.395822  309.791644   
2016-07-13 08:30:00   650.961219   744.000000  221.854294  333.562881   
2016-07-13 09:00:00   619.946272   744.000000  226.400768  341.190789   
2016-07-13 09:30:00   582.289575   744.000000  226.220077  323.081081   
2016-07-13 10:00:00   578.363985   744.000000  228.418719  316.256158   
2016-07-13 10:30:00   540.431718   744.000000  234.367291  345.487885   
2016-07-13 11:00:00   510.024256   744.000000  236.076626  372.024256   
2016-07-13 11:30:00   514.474863   744.000000  239.044262  388.309836   
2016-07-13 12:00:00   506.000000   743.664456  243.348811  413.744057   
2016-07-13 12:30:00   502.357697   743.000000  247.668882  427.066445   
2016-07-13 13:00:00   489.135388   743.000000  249.280682  413.516236   
2016-07-13 13:30:00   486.062023   743.000000  252.246060  413.845450   
2016-07-13 14:00:00   519.427890   743.000000  246.784267  374.462455   
2016-07-13 14:30:00   557.616566   743.000000  242.709270  349.546352   
2016-07-13 15:00:00   564.278571   743.000000  243.671429  353.521429   
2016-07-13 15

In [142]:
pd.set_option("display.max_rows", 10000)
w = mm94[mm94.new==537]
#w[w.data.t_air.isnull()]
#del w['new']
r = interpolaterrr(w)
r[r.data.pressure.isnull()]

Empty DataFrame
Columns: [(date_time, ), (station_id, ), (data, dampness), (data, pressure), (data, t_air), (data, t_road), (data, t_underroad), (id, dampness), (id, pressure), (id, t_air), (id, t_road), (id, t_underroad), (valid, dampness), (valid, pressure), (valid, t_air), (valid, t_road), (valid, t_underroad), (new, ), (interpol, )]
Index: []

In [139]:
r['interpol']

0        True
1       False
2        True
3       False
4        True
5       False
6        True
7       False
8        True
9       False
10       True
11      False
12       True
13      False
14       True
15      False
16       True
17      False
18       True
19      False
20       True
21      False
22       True
23      False
24       True
25      False
26       True
27      False
28       True
29      False
30       True
31       True
32       True
33      False
34       True
35       True
36      False
37       True
38      False
39       True
40      False
41       True
42      False
43       True
44      False
45       True
46      False
47       True
48      False
49       True
50      False
51       True
52      False
53       True
54      False
55       True
56      False
57       True
58      False
59       True
60      False
61       True
62      False
63       True
64      False
65       True
66      False
67       True
68      False
69       True
70       True
71    